# H01: Envelope Correlation (CCorr)

**Duration**: 55 minutes  
**Prerequisites**: B03 (Amplitude Envelope), E02 (Introduction to Hyperscanning)  
**Next**: H02 (Power Correlation)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Distinguish amplitude coupling from phase coupling
2. Define envelope correlation as Pearson correlation of amplitude envelopes
3. Implement envelope correlation computation
4. Interpret envelope correlation values (-1 to +1)
5. Understand orthogonalization to address volume conduction
6. Apply envelope correlation to hyperscanning analysis
7. Compare amplitude-based and phase-based connectivity

In [ ]:
# Required imports
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from scipy import signal
from scipy import stats
from typing import Any, Tuple

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')

# Color palette
COLORS = {
    'signal_1': '#2E86AB',
    'signal_2': '#E94F37',
    'accent': '#A23B72',
    'highlight': '#F18F01',
    'warning': '#C73E1D',
    'subject_1': '#2E86AB',
    'subject_2': '#E94F37',
}

# Random seed for reproducibility
np.random.seed(42)

---

## Section 1: Introduction — A Different Kind of Connectivity

So far in this workshop, we've focused on **phase-based connectivity metrics**: PLV, PLI, wPLI, and coherence. These metrics answer the question: *"Are oscillations aligned in time?"* They focus on **WHEN** peaks occur—whether two signals oscillate in sync.

Now we turn to a fundamentally different approach: **amplitude-based connectivity**. These metrics ask: *"Do oscillation strengths rise and fall together?"* They focus on **HOW STRONG** the oscillations are at any given moment, regardless of their phase alignment.

This distinction reflects different neural mechanisms:

- **Phase synchrony** suggests precise timing coordination between regions, often associated with active communication and information transfer
- **Amplitude correlation** suggests co-modulation of activity levels, often associated with shared engagement, arousal, or attentional states

Consider this intuition: Two brain regions might both become "more alpha-y" during relaxation. Their alpha power increases together over time. But their alpha oscillations might not be phase-locked! The peaks don't align, yet the *amount* of alpha activity rises and falls in unison.

**Why do both matter?** Phase coupling captures moment-to-moment coordination (within milliseconds), while amplitude coupling captures slower co-modulation of "engagement" (over seconds). They provide complementary views of brain connectivity.

In [ ]:
# Visualization 1: Phase coupling vs amplitude coupling

np.random.seed(42)
fs = 500.0
duration = 3.0
t = np.arange(0, duration, 1/fs)
n_samples = len(t)
freq = 10.0

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# === Left column: Phase-locked signals ===
# Signals with same phase but independent amplitude modulation
phase_lag = 0.1  # Small, consistent phase lag
env1_phase = 1 + 0.3 * np.sin(2 * np.pi * 0.3 * t)  # Independent modulation
env2_phase = 1 + 0.3 * np.sin(2 * np.pi * 0.5 * t + 1.5)  # Different modulation

x_phase = env1_phase * np.sin(2 * np.pi * freq * t)
y_phase = env2_phase * np.sin(2 * np.pi * freq * t + phase_lag)

ax1 = axes[0, 0]
ax1.plot(t, x_phase, color=COLORS['signal_1'], linewidth=1, label='Signal X', alpha=0.8)
ax1.plot(t, y_phase, color=COLORS['signal_2'], linewidth=1, label='Signal Y', alpha=0.8)
ax1.set_xlabel('Time (s)', fontsize=11)
ax1.set_ylabel('Amplitude', fontsize=11)
ax1.set_title('Phase-Locked Signals\n(peaks aligned, envelopes independent)', fontsize=12, fontweight='bold')
ax1.legend(loc='upper right')
ax1.set_xlim(0, 1)  # Zoom in

# Zoom showing phase alignment
ax2 = axes[1, 0]
t_zoom = (t >= 0.3) & (t <= 0.6)
ax2.plot(t[t_zoom], x_phase[t_zoom], color=COLORS['signal_1'], linewidth=2, label='Signal X')
ax2.plot(t[t_zoom], y_phase[t_zoom], color=COLORS['signal_2'], linewidth=2, label='Signal Y')
# Mark peaks
for i in range(len(t[t_zoom]) - 1):
    idx = np.where(t_zoom)[0][i]
    if x_phase[idx] > x_phase[idx-1] and x_phase[idx] > x_phase[idx+1] and x_phase[idx] > 0.5:
        ax2.axvline(t[idx], color=COLORS['signal_1'], alpha=0.3, linestyle='--')
ax2.set_xlabel('Time (s)', fontsize=11)
ax2.set_ylabel('Amplitude', fontsize=11)
ax2.set_title('Zoomed: Peaks are aligned (phase coupling)', fontsize=11)
ax2.legend(loc='upper right')

# === Right column: Amplitude-correlated signals ===
# Signals with correlated envelopes but random phases
shared_env = 1 + 0.5 * np.sin(2 * np.pi * 0.4 * t)  # Shared slow modulation
phase_x = 2 * np.pi * freq * t
phase_y = 2 * np.pi * freq * t + np.cumsum(0.5 * np.random.randn(n_samples)) / fs  # Random phase drift

x_amp = shared_env * np.sin(phase_x)
y_amp = shared_env * np.sin(phase_y)

ax3 = axes[0, 1]
ax3.plot(t, x_amp, color=COLORS['signal_1'], linewidth=1, label='Signal X', alpha=0.8)
ax3.plot(t, y_amp, color=COLORS['signal_2'], linewidth=1, label='Signal Y', alpha=0.8)
ax3.plot(t, shared_env, color=COLORS['highlight'], linewidth=2, linestyle='--', label='Shared envelope')
ax3.plot(t, -shared_env, color=COLORS['highlight'], linewidth=2, linestyle='--')
ax3.set_xlabel('Time (s)', fontsize=11)
ax3.set_ylabel('Amplitude', fontsize=11)
ax3.set_title('Amplitude-Correlated Signals\n(envelopes coupled, phases independent)', fontsize=12, fontweight='bold')
ax3.legend(loc='upper right')
ax3.set_xlim(0, 3)

# Zoom showing phase misalignment
ax4 = axes[1, 1]
ax4.plot(t[t_zoom], x_amp[t_zoom], color=COLORS['signal_1'], linewidth=2, label='Signal X')
ax4.plot(t[t_zoom], y_amp[t_zoom], color=COLORS['signal_2'], linewidth=2, label='Signal Y')
ax4.set_xlabel('Time (s)', fontsize=11)
ax4.set_ylabel('Amplitude', fontsize=11)
ax4.set_title('Zoomed: Peaks NOT aligned (no phase coupling)', fontsize=11)
ax4.legend(loc='upper right')

plt.suptitle('Phase Coupling vs Amplitude Coupling: Different Phenomena', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("Left: High PHASE coupling (peaks aligned), low amplitude coupling")
print("Right: High AMPLITUDE coupling (envelopes correlated), low phase coupling")
print("\n💡 These are different dimensions of connectivity!")

---

## Section 2: Recall — The Amplitude Envelope

From B03, we learned that the **amplitude envelope** represents the instantaneous oscillation strength. It's computed using the Hilbert transform:

$$A(t) = |z(t)| = \sqrt{x(t)^2 + \hat{x}(t)^2}$$

where $z(t)$ is the analytic signal and $\hat{x}(t)$ is the Hilbert transform of $x(t)$.

The envelope varies more slowly than the oscillation itself—it captures "how strong" the oscillation is at each moment, abstracting away the rapid up-down fluctuations.

In [ ]:
# Helper functions

def bandpass_filter(
    data: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    order: int = 4
) -> NDArray[np.float64]:
    """
    Apply bandpass filter to signal.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        Input signal
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
    order : int, optional
        Filter order (default: 4)
        
    Returns
    -------
    NDArray[np.float64]
        Filtered signal
    """
    nyq = fs / 2
    low, high = band[0] / nyq, band[1] / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.filtfilt(b, a, data, axis=-1)


def extract_envelope(
    data: NDArray[np.float64]
) -> NDArray[np.float64]:
    """
    Extract amplitude envelope using Hilbert transform.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        Input signal
        
    Returns
    -------
    NDArray[np.float64]
        Amplitude envelope
    """
    analytic = signal.hilbert(data, axis=-1)
    return np.abs(analytic)


def extract_phase(
    data: NDArray[np.float64]
) -> NDArray[np.float64]:
    """
    Extract instantaneous phase using Hilbert transform.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        Input signal
        
    Returns
    -------
    NDArray[np.float64]
        Instantaneous phase in radians
    """
    analytic = signal.hilbert(data, axis=-1)
    return np.angle(analytic)

In [ ]:
# Visualization 2: Signal with envelope

np.random.seed(42)
fs = 500.0
duration = 4.0
t = np.arange(0, duration, 1/fs)

# Create amplitude-modulated signal
modulation = 1 + 0.6 * np.sin(2 * np.pi * 0.3 * t)  # Slow modulation
carrier = np.sin(2 * np.pi * 10 * t)  # 10 Hz oscillation
sig = modulation * carrier + 0.2 * np.random.randn(len(t))

# Filter and extract envelope
sig_filt = bandpass_filter(sig, fs, (8, 12))
envelope = extract_envelope(sig_filt)

fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(t, sig_filt, color=COLORS['signal_1'], linewidth=1, alpha=0.7, label='Filtered signal (8-12 Hz)')
ax.plot(t, envelope, color=COLORS['highlight'], linewidth=2.5, label='Amplitude envelope')
ax.plot(t, -envelope, color=COLORS['highlight'], linewidth=2.5)

# Annotations
ax.annotate('High amplitude\n(strong oscillation)', xy=(1.7, envelope[int(1.7*fs)]), 
            xytext=(2.2, 1.5), fontsize=10,
            arrowprops=dict(arrowstyle='->', color='black'))
ax.annotate('Low amplitude\n(weak oscillation)', xy=(3.3, envelope[int(3.3*fs)]), 
            xytext=(3.5, 1.3), fontsize=10,
            arrowprops=dict(arrowstyle='->', color='black'))

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Amplitude', fontsize=12)
ax.set_title('Amplitude Envelope: Instantaneous Oscillation Strength', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=11)
ax.set_xlim(0, duration)

plt.tight_layout()
plt.show()

---

## Section 3: Envelope Correlation Definition

**Envelope correlation** (also called **Amplitude Envelope Correlation, AEC**) is simply the Pearson correlation between two amplitude envelopes:

$$CCorr = \frac{Cov(A_x, A_y)}{\sigma_{A_x} \cdot \sigma_{A_y}} = Pearson(A_x, A_y)$$

### Properties:

- **Range**: -1 to +1 (unlike phase metrics which are 0 to 1!)
- **CCorr = +1**: Envelopes perfectly positively correlated (rise and fall together)
- **CCorr = -1**: Envelopes perfectly anti-correlated (one up when other down)
- **CCorr = 0**: No linear relationship between envelopes

### Interpretation:

- **Positive**: Regions "activate" together—when one has strong oscillations, so does the other
- **Negative**: Reciprocal relationship—one activates when the other deactivates (less common)
- **Zero**: Independent amplitude fluctuations

In [ ]:
# Visualization 3: Three CCorr scenarios

np.random.seed(42)
fs = 500.0
duration = 5.0
t = np.arange(0, duration, 1/fs)
n_samples = len(t)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# === Scenario 1: High positive correlation ===
shared_mod = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
env1_pos = shared_mod + 0.1 * np.random.randn(n_samples)
env2_pos = shared_mod + 0.1 * np.random.randn(n_samples)
ccorr_pos = np.corrcoef(env1_pos, env2_pos)[0, 1]

ax1 = axes[0]
ax1.plot(t, env1_pos, color=COLORS['signal_1'], linewidth=2, label='Envelope X')
ax1.plot(t, env2_pos, color=COLORS['signal_2'], linewidth=2, label='Envelope Y')
ax1.set_xlabel('Time (s)', fontsize=11)
ax1.set_ylabel('Amplitude', fontsize=11)
ax1.set_title(f'Positive Correlation\nCCorr = {ccorr_pos:.2f}', fontsize=12, fontweight='bold')
ax1.legend(loc='upper right')

# === Scenario 2: No correlation ===
env1_zero = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
env2_zero = 1 + 0.5 * np.sin(2 * np.pi * 0.5 * t + 2.0)  # Different frequency and phase
ccorr_zero = np.corrcoef(env1_zero, env2_zero)[0, 1]

ax2 = axes[1]
ax2.plot(t, env1_zero, color=COLORS['signal_1'], linewidth=2, label='Envelope X')
ax2.plot(t, env2_zero, color=COLORS['signal_2'], linewidth=2, label='Envelope Y')
ax2.set_xlabel('Time (s)', fontsize=11)
ax2.set_ylabel('Amplitude', fontsize=11)
ax2.set_title(f'No Correlation\nCCorr = {ccorr_zero:.2f}', fontsize=12, fontweight='bold')
ax2.legend(loc='upper right')

# === Scenario 3: Negative correlation ===
env1_neg = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
env2_neg = 1 - 0.5 * np.sin(2 * np.pi * 0.3 * t) + 0.1 * np.random.randn(n_samples)  # Inverted
ccorr_neg = np.corrcoef(env1_neg, env2_neg)[0, 1]

ax3 = axes[2]
ax3.plot(t, env1_neg, color=COLORS['signal_1'], linewidth=2, label='Envelope X')
ax3.plot(t, env2_neg, color=COLORS['signal_2'], linewidth=2, label='Envelope Y')
ax3.set_xlabel('Time (s)', fontsize=11)
ax3.set_ylabel('Amplitude', fontsize=11)
ax3.set_title(f'Negative Correlation\nCCorr = {ccorr_neg:.2f}', fontsize=12, fontweight='bold')
ax3.legend(loc='upper right')

plt.suptitle('Envelope Correlation Captures Amplitude Co-Modulation', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

## Section 4: Implementing Envelope Correlation

The computation pipeline is straightforward:

1. **Band-pass filter** both signals (frequency of interest)
2. **Extract amplitude envelopes** (Hilbert transform)
3. (Optional) **Low-pass filter envelopes** to focus on slow fluctuations
4. **Compute Pearson correlation** between envelopes

**Key considerations**:
- Must filter first! (Envelope of broadband signal isn't meaningful for connectivity)
- Optional envelope smoothing removes fast fluctuations
- Use `scipy.stats.pearsonr` for correlation

In [ ]:
def smooth_envelope(
    envelope: NDArray[np.float64],
    fs: float,
    cutoff: float = 1.0
) -> NDArray[np.float64]:
    """
    Low-pass filter envelope for slow fluctuations only.
    
    Parameters
    ----------
    envelope : NDArray[np.float64]
        Amplitude envelope
    fs : float
        Sampling frequency in Hz
    cutoff : float, optional
        Low-pass cutoff frequency in Hz (default: 1.0)
        
    Returns
    -------
    NDArray[np.float64]
        Smoothed envelope
    """
    nyq = fs / 2
    normalized_cutoff = cutoff / nyq
    
    # Ensure cutoff is valid
    if normalized_cutoff >= 1:
        return envelope
    
    b, a = signal.butter(4, normalized_cutoff, btype='low')
    return signal.filtfilt(b, a, envelope, axis=-1)


def compute_envelope_correlation_from_envelopes(
    env_x: NDArray[np.float64],
    env_y: NDArray[np.float64]
) -> float:
    """
    Compute envelope correlation from pre-computed envelopes.
    
    Parameters
    ----------
    env_x : NDArray[np.float64]
        Amplitude envelope of signal x
    env_y : NDArray[np.float64]
        Amplitude envelope of signal y
        
    Returns
    -------
    float
        Pearson correlation coefficient in range [-1, 1]
    """
    # Handle constant signals
    if np.std(env_x) == 0 or np.std(env_y) == 0:
        return 0.0
    
    corr, _ = stats.pearsonr(env_x, env_y)
    return float(corr)


def compute_envelope_correlation(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    filter_order: int = 4,
    envelope_lowpass: float | None = None
) -> float:
    """
    Compute envelope correlation between two signals.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        First signal
    y : NDArray[np.float64]
        Second signal
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
    filter_order : int, optional
        Butterworth filter order (default: 4)
    envelope_lowpass : float, optional
        Optional low-pass cutoff for envelope smoothing in Hz
        
    Returns
    -------
    float
        Envelope correlation in range [-1, 1]
    """
    # Bandpass filter
    x_filt = bandpass_filter(x, fs, band, filter_order)
    y_filt = bandpass_filter(y, fs, band, filter_order)
    
    # Extract envelopes
    env_x = extract_envelope(x_filt)
    env_y = extract_envelope(y_filt)
    
    # Optional smoothing
    if envelope_lowpass is not None:
        env_x = smooth_envelope(env_x, fs, envelope_lowpass)
        env_y = smooth_envelope(env_y, fs, envelope_lowpass)
    
    return compute_envelope_correlation_from_envelopes(env_x, env_y)

In [ ]:
# Visualization 4: Computation pipeline

np.random.seed(42)
fs = 500.0
duration = 4.0
t = np.arange(0, duration, 1/fs)
n_samples = len(t)

# Create two signals with correlated amplitude modulation
shared_mod = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
x_raw = shared_mod * np.sin(2 * np.pi * 10 * t) + 0.3 * np.random.randn(n_samples)
y_raw = shared_mod * np.sin(2 * np.pi * 10 * t + np.pi/3) + 0.3 * np.random.randn(n_samples)

# Add some broadband noise
x_raw += 0.2 * np.random.randn(n_samples)
y_raw += 0.2 * np.random.randn(n_samples)

# Step by step
x_filt = bandpass_filter(x_raw, fs, (8, 12))
y_filt = bandpass_filter(y_raw, fs, (8, 12))
env_x = extract_envelope(x_filt)
env_y = extract_envelope(y_filt)
ccorr = compute_envelope_correlation_from_envelopes(env_x, env_y)

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Step 1: Raw signals
ax1 = axes[0, 0]
ax1.plot(t, x_raw, color=COLORS['signal_1'], linewidth=1, alpha=0.8, label='X')
ax1.plot(t, y_raw, color=COLORS['signal_2'], linewidth=1, alpha=0.8, label='Y')
ax1.set_xlabel('Time (s)', fontsize=10)
ax1.set_ylabel('Amplitude', fontsize=10)
ax1.set_title('Step 1: Raw Signals', fontsize=11, fontweight='bold')
ax1.legend(loc='upper right')
ax1.set_xlim(0, 2)

# Step 2: Bandpass filter
ax2 = axes[0, 1]
ax2.plot(t, x_filt, color=COLORS['signal_1'], linewidth=1, alpha=0.8, label='X filtered')
ax2.plot(t, y_filt, color=COLORS['signal_2'], linewidth=1, alpha=0.8, label='Y filtered')
ax2.set_xlabel('Time (s)', fontsize=10)
ax2.set_ylabel('Amplitude', fontsize=10)
ax2.set_title('Step 2: Bandpass Filter (8-12 Hz)', fontsize=11, fontweight='bold')
ax2.legend(loc='upper right')
ax2.set_xlim(0, 2)

# Step 3: Extract envelopes (Hilbert)
ax3 = axes[0, 2]
ax3.plot(t, x_filt, color=COLORS['signal_1'], linewidth=1, alpha=0.3)
ax3.plot(t, y_filt, color=COLORS['signal_2'], linewidth=1, alpha=0.3)
ax3.plot(t, env_x, color=COLORS['signal_1'], linewidth=2, label='Envelope X')
ax3.plot(t, env_y, color=COLORS['signal_2'], linewidth=2, label='Envelope Y')
ax3.set_xlabel('Time (s)', fontsize=10)
ax3.set_ylabel('Amplitude', fontsize=10)
ax3.set_title('Step 3: Extract Envelopes (Hilbert)', fontsize=11, fontweight='bold')
ax3.legend(loc='upper right')
ax3.set_xlim(0, 2)

# Step 4: Envelopes only
ax4 = axes[1, 0]
ax4.plot(t, env_x, color=COLORS['signal_1'], linewidth=2, label='Envelope X')
ax4.plot(t, env_y, color=COLORS['signal_2'], linewidth=2, label='Envelope Y')
ax4.set_xlabel('Time (s)', fontsize=10)
ax4.set_ylabel('Envelope', fontsize=10)
ax4.set_title('Step 4: Compare Envelopes', fontsize=11, fontweight='bold')
ax4.legend(loc='upper right')

# Step 5: Scatter plot
ax5 = axes[1, 1]
ax5.scatter(env_x, env_y, alpha=0.3, color=COLORS['accent'], s=5)
# Fit line
z = np.polyfit(env_x, env_y, 1)
p = np.poly1d(z)
x_line = np.linspace(env_x.min(), env_x.max(), 100)
ax5.plot(x_line, p(x_line), color=COLORS['highlight'], linewidth=2, label=f'r = {ccorr:.3f}')
ax5.set_xlabel('Envelope X', fontsize=10)
ax5.set_ylabel('Envelope Y', fontsize=10)
ax5.set_title('Step 5: Pearson Correlation', fontsize=11, fontweight='bold')
ax5.legend(loc='upper left')

# Final result
ax6 = axes[1, 2]
ax6.bar(['CCorr'], [ccorr], color=COLORS['highlight'], edgecolor='white', linewidth=2, width=0.5)
ax6.text(0, ccorr + 0.05, f'{ccorr:.3f}', ha='center', fontsize=14, fontweight='bold')
ax6.set_ylabel('Correlation', fontsize=10)
ax6.set_title('Result: Envelope Correlation', fontsize=11, fontweight='bold')
ax6.set_ylim(-1, 1)
ax6.axhline(0, color='black', linestyle='--', alpha=0.3)

plt.suptitle('Envelope Correlation Computation Pipeline', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

## Section 5: Envelope Correlation with Controlled Examples

Let's build intuition with synthetic signals where we control the amplitude correlation.

In [ ]:
def generate_amplitude_correlated_signals(
    n_samples: int,
    fs: float,
    frequency: float,
    correlation_target: float,
    modulation_freq: float = 0.5,
    seed: int | None = None
) -> Tuple[NDArray[np.float64], NDArray[np.float64]]:
    """
    Generate two signals with specified envelope correlation.
    
    Parameters
    ----------
    n_samples : int
        Number of samples
    fs : float
        Sampling frequency in Hz
    frequency : float
        Carrier frequency in Hz
    correlation_target : float
        Desired envelope correlation (-1 to 1)
    modulation_freq : float, optional
        Envelope modulation frequency in Hz (default: 0.5)
    seed : int, optional
        Random seed for reproducibility
        
    Returns
    -------
    Tuple[NDArray[np.float64], NDArray[np.float64]]
        Two signals with correlated envelopes
    """
    if seed is not None:
        np.random.seed(seed)
    
    t = np.arange(n_samples) / fs
    
    # Shared and independent envelope components
    shared = np.sin(2 * np.pi * modulation_freq * t)
    indep_x = np.sin(2 * np.pi * modulation_freq * 1.3 * t + np.random.uniform(0, 2*np.pi))
    indep_y = np.sin(2 * np.pi * modulation_freq * 1.7 * t + np.random.uniform(0, 2*np.pi))
    
    # Mix shared and independent based on target correlation
    # correlation_target ≈ shared_weight^2 (approximately)
    if correlation_target >= 0:
        shared_weight = np.sqrt(abs(correlation_target))
        indep_weight = np.sqrt(1 - abs(correlation_target))
        env_x = 1 + 0.4 * (shared_weight * shared + indep_weight * indep_x)
        env_y = 1 + 0.4 * (shared_weight * shared + indep_weight * indep_y)
    else:
        shared_weight = np.sqrt(abs(correlation_target))
        indep_weight = np.sqrt(1 - abs(correlation_target))
        env_x = 1 + 0.4 * (shared_weight * shared + indep_weight * indep_x)
        env_y = 1 + 0.4 * (-shared_weight * shared + indep_weight * indep_y)  # Inverted
    
    # Ensure positive envelopes
    env_x = np.maximum(env_x, 0.2)
    env_y = np.maximum(env_y, 0.2)
    
    # Generate signals with independent phases
    phase_x = 2 * np.pi * frequency * t
    phase_y = 2 * np.pi * frequency * t + np.random.uniform(0, 2*np.pi)
    
    x = env_x * np.sin(phase_x) + 0.1 * np.random.randn(n_samples)
    y = env_y * np.sin(phase_y) + 0.1 * np.random.randn(n_samples)
    
    return x, y

In [ ]:
# Visualization 5: Grid of examples (2×2)

fs = 500.0
n_samples = 5000
frequency = 10.0
band = (8, 12)

scenarios = [
    {'target': 0.9, 'title': 'High Positive'},
    {'target': 0.0, 'title': 'No Correlation'},
    {'target': 0.5, 'title': 'Moderate Positive'},
    {'target': -0.6, 'title': 'Negative'}
]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for ax, scenario in zip(axes, scenarios):
    x, y = generate_amplitude_correlated_signals(
        n_samples, fs, frequency, scenario['target'], seed=42
    )
    
    # Filter and extract envelopes
    x_filt = bandpass_filter(x, fs, band)
    y_filt = bandpass_filter(y, fs, band)
    env_x = extract_envelope(x_filt)
    env_y = extract_envelope(y_filt)
    
    # Actual correlation
    actual_ccorr = compute_envelope_correlation_from_envelopes(env_x, env_y)
    
    t = np.arange(n_samples) / fs
    
    # Plot signals (faded) and envelopes
    ax.plot(t, x_filt, color=COLORS['signal_1'], linewidth=0.5, alpha=0.3)
    ax.plot(t, y_filt, color=COLORS['signal_2'], linewidth=0.5, alpha=0.3)
    ax.plot(t, env_x, color=COLORS['signal_1'], linewidth=2, label='Env X')
    ax.plot(t, env_y, color=COLORS['signal_2'], linewidth=2, label='Env Y')
    
    ax.set_xlabel('Time (s)', fontsize=10)
    ax.set_ylabel('Amplitude', fontsize=10)
    ax.set_title(f"{scenario['title']}\nTarget: {scenario['target']:.1f}, Actual CCorr: {actual_ccorr:.2f}", 
                 fontsize=11, fontweight='bold')
    ax.legend(loc='upper right')
    ax.set_xlim(0, 6)

plt.suptitle('Envelope Correlation Examples', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Visualization 6: CCorr vs shared modulation

shared_proportions = np.linspace(0, 1, 20)
measured_ccorrs = []

for prop in shared_proportions:
    x, y = generate_amplitude_correlated_signals(
        n_samples=10000, fs=500.0, frequency=10.0, 
        correlation_target=prop, seed=42
    )
    ccorr = compute_envelope_correlation(x, y, 500.0, (8, 12))
    measured_ccorrs.append(ccorr)

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(shared_proportions, measured_ccorrs, 'o-', color=COLORS['signal_1'], 
        linewidth=2, markersize=8, label='Measured CCorr')
ax.plot([0, 1], [0, 1], '--', color='gray', linewidth=2, label='Ideal (y=x)')

ax.set_xlabel('Target Correlation', fontsize=12)
ax.set_ylabel('Measured Envelope Correlation', fontsize=12)
ax.set_title('Envelope Correlation Tracks Shared Modulation', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=11)
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(-0.1, 1.1)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Section 6: The Volume Conduction Problem (Again!)

Volume conduction affects amplitude correlation too—and it's arguably **worse** than for phase metrics!

**The problem**: When the same neural source projects to two electrodes, both see nearly identical signals with the same envelope. This creates artificially high CCorr.

**Why it's worse than for phase**: With phase metrics like PLI/wPLI, we could exploit the fact that volume conduction creates zero-lag relationships. But for amplitude, the zero-lag component IS the signal we're trying to measure. There's no simple "lag" to filter out.

**The solution**: **Orthogonalization** (Hipp et al., 2012)

In [ ]:
# Visualization 7: Volume conduction problem

np.random.seed(42)
fs = 500.0
duration = 5.0
t = np.arange(0, duration, 1/fs)
n_samples = len(t)

# Single source with amplitude modulation
modulation = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
source = modulation * np.sin(2 * np.pi * 10 * t)

# Two electrodes picking up the same source (volume conduction)
x_vc = source + 0.15 * np.random.randn(n_samples)
y_vc = 0.8 * source + 0.15 * np.random.randn(n_samples)  # Slightly attenuated

# Filter and extract envelopes
x_filt = bandpass_filter(x_vc, fs, (8, 12))
y_filt = bandpass_filter(y_vc, fs, (8, 12))
env_x = extract_envelope(x_filt)
env_y = extract_envelope(y_filt)

ccorr_vc = compute_envelope_correlation_from_envelopes(env_x, env_y)

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Panel 1: The scenario
ax1 = axes[0, 0]
ax1.plot(t, source, color=COLORS['accent'], linewidth=2, label='True source')
ax1.set_xlabel('Time (s)', fontsize=11)
ax1.set_ylabel('Amplitude', fontsize=11)
ax1.set_title('Single Neural Source', fontsize=12, fontweight='bold')
ax1.legend(loc='upper right')
ax1.set_xlim(0, 3)

# Panel 2: Signals at two electrodes
ax2 = axes[0, 1]
ax2.plot(t, x_filt, color=COLORS['signal_1'], linewidth=1, alpha=0.8, label='Electrode 1')
ax2.plot(t, y_filt, color=COLORS['signal_2'], linewidth=1, alpha=0.8, label='Electrode 2')
ax2.set_xlabel('Time (s)', fontsize=11)
ax2.set_ylabel('Amplitude', fontsize=11)
ax2.set_title('Volume Conduction: Same source at two electrodes', fontsize=12, fontweight='bold')
ax2.legend(loc='upper right')
ax2.set_xlim(0, 3)

# Panel 3: Envelopes
ax3 = axes[1, 0]
ax3.plot(t, env_x, color=COLORS['signal_1'], linewidth=2, label='Envelope 1')
ax3.plot(t, env_y, color=COLORS['signal_2'], linewidth=2, label='Envelope 2')
ax3.set_xlabel('Time (s)', fontsize=11)
ax3.set_ylabel('Envelope', fontsize=11)
ax3.set_title('Nearly Identical Envelopes!', fontsize=12, fontweight='bold')
ax3.legend(loc='upper right')

# Panel 4: Scatter + result
ax4 = axes[1, 1]
ax4.scatter(env_x, env_y, alpha=0.3, color=COLORS['warning'], s=5)
z = np.polyfit(env_x, env_y, 1)
p = np.poly1d(z)
x_line = np.linspace(env_x.min(), env_x.max(), 100)
ax4.plot(x_line, p(x_line), color='black', linewidth=2)
ax4.set_xlabel('Envelope 1', fontsize=11)
ax4.set_ylabel('Envelope 2', fontsize=11)
ax4.set_title(f'Spurious High Correlation!\nCCorr = {ccorr_vc:.3f}', fontsize=12, fontweight='bold')

plt.suptitle('Volume Conduction Inflates Envelope Correlation', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"⚠️ CCorr = {ccorr_vc:.3f} — This is SPURIOUS due to volume conduction!")

---

## Section 7: Orthogonalized Envelope Correlation

**Orthogonalization** (Hipp et al., 2012) removes the zero-lag component that volume conduction creates.

### The Method:

1. Compute analytic signals $z_x(t)$ and $z_y(t)$
2. Orthogonalize: Remove the component of $z_y$ that's in phase with $z_x$:
   $$z_{y\perp}(t) = \text{Im}(z_y(t) \cdot e^{-i\phi_x(t)})$$
3. Compute envelopes from orthogonalized signals
4. Correlate the orthogonalized envelopes

### Properties:

- Still ranges -1 to +1
- **Symmetric version**: Average correlations in both directions ($y\perp x$ and $x\perp y$)
- More conservative than standard CCorr
- Removes volume conduction artifacts

In [ ]:
def orthogonalize_signals(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> Tuple[NDArray[np.float64], NDArray[np.float64]]:
    """
    Orthogonalize signals to remove zero-lag component.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        First signal
    y : NDArray[np.float64]
        Second signal
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
        
    Returns
    -------
    Tuple[NDArray[np.float64], NDArray[np.float64]]
        (y_orthogonalized_to_x, x_orthogonalized_to_y)
        
    Notes
    -----
    Implements Hipp et al. (2012) orthogonalization:
    y_orth = Im(z_y * exp(-i * phase_x))
    This removes the component of y that's in phase with x.
    """
    # Bandpass filter
    x_filt = bandpass_filter(x, fs, band)
    y_filt = bandpass_filter(y, fs, band)
    
    # Analytic signals
    z_x = signal.hilbert(x_filt)
    z_y = signal.hilbert(y_filt)
    
    # Extract phases
    phase_x = np.angle(z_x)
    phase_y = np.angle(z_y)
    
    # Orthogonalize y with respect to x
    # y_orth = Im(z_y * exp(-i * phase_x))
    y_orth = np.imag(z_y * np.exp(-1j * phase_x))
    
    # Orthogonalize x with respect to y
    x_orth = np.imag(z_x * np.exp(-1j * phase_y))
    
    return y_orth, x_orth


def compute_orthogonalized_envelope_correlation(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    symmetric: bool = True
) -> float:
    """
    Compute envelope correlation with orthogonalization (AEC-c).
    
    Parameters
    ----------
    x : NDArray[np.float64]
        First signal
    y : NDArray[np.float64]
        Second signal
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
    symmetric : bool, optional
        If True, average correlations in both directions (default: True)
        
    Returns
    -------
    float
        Orthogonalized envelope correlation in range [-1, 1]
    """
    # Bandpass filter
    x_filt = bandpass_filter(x, fs, band)
    y_filt = bandpass_filter(y, fs, band)
    
    # Analytic signals
    z_x = signal.hilbert(x_filt)
    z_y = signal.hilbert(y_filt)
    
    # Extract phases and envelopes
    phase_x = np.angle(z_x)
    phase_y = np.angle(z_y)
    env_x = np.abs(z_x)
    env_y = np.abs(z_y)
    
    # Orthogonalize y with respect to x and get its envelope
    y_orth = np.imag(z_y * np.exp(-1j * phase_x))
    env_y_orth = np.abs(y_orth)
    
    # Correlate original x envelope with orthogonalized y envelope
    corr_xy = compute_envelope_correlation_from_envelopes(env_x, env_y_orth)
    
    if symmetric:
        # Also compute in the other direction
        x_orth = np.imag(z_x * np.exp(-1j * phase_y))
        env_x_orth = np.abs(x_orth)
        corr_yx = compute_envelope_correlation_from_envelopes(env_y, env_x_orth)
        
        # Average both directions
        return (corr_xy + corr_yx) / 2
    
    return corr_xy

In [ ]:
# Visualization 8: Orthogonalization effect

# Using the volume conduction signals from before
y_orth, x_orth = orthogonalize_signals(x_vc, y_vc, fs, (8, 12))

# Get envelopes
env_y_orth = extract_envelope(y_orth)
env_x_orth = extract_envelope(x_orth)

# Compute correlations
ccorr_standard = compute_envelope_correlation(x_vc, y_vc, fs, (8, 12))
ccorr_orth = compute_orthogonalized_envelope_correlation(x_vc, y_vc, fs, (8, 12))

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Original envelopes
ax1 = axes[0, 0]
ax1.plot(t, env_x, color=COLORS['signal_1'], linewidth=2, label='Envelope X')
ax1.plot(t, env_y, color=COLORS['signal_2'], linewidth=2, label='Envelope Y')
ax1.set_xlabel('Time (s)', fontsize=11)
ax1.set_ylabel('Envelope', fontsize=11)
ax1.set_title(f'Original Envelopes\nCCorr = {ccorr_standard:.3f} (inflated!)', fontsize=12, fontweight='bold')
ax1.legend(loc='upper right')

# Orthogonalized signals
ax2 = axes[0, 1]
ax2.plot(t, y_orth, color=COLORS['signal_2'], linewidth=1, alpha=0.8, label='Y orthogonalized to X')
ax2.set_xlabel('Time (s)', fontsize=11)
ax2.set_ylabel('Amplitude', fontsize=11)
ax2.set_title('Orthogonalized Signal\n(zero-lag component removed)', fontsize=12, fontweight='bold')
ax2.legend(loc='upper right')

# Orthogonalized envelopes
ax3 = axes[1, 0]
ax3.plot(t, env_x, color=COLORS['signal_1'], linewidth=2, label='Envelope X (original)')
ax3.plot(t, env_y_orth, color=COLORS['signal_2'], linewidth=2, label='Envelope Y (orthogonalized)')
ax3.set_xlabel('Time (s)', fontsize=11)
ax3.set_ylabel('Envelope', fontsize=11)
ax3.set_title('After Orthogonalization', fontsize=12, fontweight='bold')
ax3.legend(loc='upper right')

# Comparison
ax4 = axes[1, 1]
bars = ax4.bar(['Standard\nCCorr', 'Orthogonalized\nCCorr'], [ccorr_standard, ccorr_orth],
               color=[COLORS['warning'], COLORS['signal_1']], edgecolor='white', linewidth=2)
ax4.axhline(0, color='black', linestyle='-', linewidth=1)
for bar, val in zip(bars, [ccorr_standard, ccorr_orth]):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
             f'{val:.3f}', ha='center', fontsize=12, fontweight='bold')
ax4.set_ylabel('Correlation', fontsize=11)
ax4.set_title('Volume Conduction Corrected!', fontsize=12, fontweight='bold')
ax4.set_ylim(-0.2, 1.1)

plt.suptitle('Orthogonalization Removes Zero-Lag (Volume Conduction) Contribution', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"Standard CCorr: {ccorr_standard:.3f} (spuriously high)")
print(f"Orthogonalized CCorr: {ccorr_orth:.3f} (corrected)")
print(f"\n✅ Orthogonalization successfully removed the volume conduction artifact!")

In [ ]:
# Visualization 9: Comparison bar chart - standard vs orthogonalized

np.random.seed(42)
fs = 500.0
n_samples = 10000
t = np.arange(n_samples) / fs
band = (8, 12)

# Volume conduction scenario
source_vc = (1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)) * np.sin(2 * np.pi * 10 * t)
x_vc = source_vc + 0.1 * np.random.randn(n_samples)
y_vc = 0.9 * source_vc + 0.1 * np.random.randn(n_samples)

# True connection scenario (with lag)
mod_true = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
x_true = mod_true * np.sin(2 * np.pi * 10 * t) + 0.2 * np.random.randn(n_samples)
y_true = mod_true * np.sin(2 * np.pi * 10 * t + np.pi/4) + 0.2 * np.random.randn(n_samples)  # Phase lag

# Compute correlations
ccorr_vc_std = compute_envelope_correlation(x_vc, y_vc, fs, band)
ccorr_vc_orth = compute_orthogonalized_envelope_correlation(x_vc, y_vc, fs, band)
ccorr_true_std = compute_envelope_correlation(x_true, y_true, fs, band)
ccorr_true_orth = compute_orthogonalized_envelope_correlation(x_true, y_true, fs, band)

fig, ax = plt.subplots(figsize=(10, 6))

x_pos = np.array([0, 1, 3, 4])
values = [ccorr_vc_std, ccorr_vc_orth, ccorr_true_std, ccorr_true_orth]
colors_bar = [COLORS['warning'], COLORS['signal_1'], COLORS['warning'], COLORS['signal_1']]
labels = ['Standard', 'Orthogonalized', 'Standard', 'Orthogonalized']

bars = ax.bar(x_pos, values, color=colors_bar, edgecolor='white', linewidth=2, width=0.8)

for bar, val in zip(bars, values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.03,
            f'{val:.3f}', ha='center', fontsize=11, fontweight='bold')

ax.set_xticks([0.5, 3.5])
ax.set_xticklabels(['Volume Conduction\n(Should be LOW)', 'True Connection\n(Should remain HIGH)'], fontsize=11)
ax.set_ylabel('Envelope Correlation', fontsize=12)
ax.set_title('Orthogonalization Distinguishes True from Spurious Connectivity', fontsize=14, fontweight='bold')
ax.set_ylim(0, 1.1)

# Legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=COLORS['warning'], label='Standard CCorr'),
                   Patch(facecolor=COLORS['signal_1'], label='Orthogonalized CCorr')]
ax.legend(handles=legend_elements, loc='upper right', fontsize=11)

plt.tight_layout()
plt.show()

print("Volume Conduction:")
print(f"  Standard: {ccorr_vc_std:.3f} → Orthogonalized: {ccorr_vc_orth:.3f} (correctly reduced)")
print(f"\nTrue Connection:")
print(f"  Standard: {ccorr_true_std:.3f} → Orthogonalized: {ccorr_true_orth:.3f} (remains high)")

---

## Section 8: Time-Resolved Envelope Correlation

Static CCorr gives a single value for the entire recording. But amplitude coupling may change over time! We can compute **dynamic CCorr** using sliding windows.

In [ ]:
def compute_envelope_correlation_timeseries(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    window_sec: float = 2.0,
    overlap: float = 0.5,
    orthogonalize: bool = True
) -> Tuple[NDArray[np.float64], NDArray[np.float64]]:
    """
    Compute time-resolved envelope correlation.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        First signal
    y : NDArray[np.float64]
        Second signal
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
    window_sec : float, optional
        Window length in seconds (default: 2.0)
    overlap : float, optional
        Overlap fraction 0-1 (default: 0.5)
    orthogonalize : bool, optional
        Whether to use orthogonalization (default: True)
        
    Returns
    -------
    Tuple[NDArray[np.float64], NDArray[np.float64]]
        (time_centers, ccorr_values)
    """
    window_samples = int(window_sec * fs)
    step_samples = int(window_samples * (1 - overlap))
    
    # Filter once
    x_filt = bandpass_filter(x, fs, band)
    y_filt = bandpass_filter(y, fs, band)
    
    # Analytic signals
    z_x = signal.hilbert(x_filt)
    z_y = signal.hilbert(y_filt)
    
    n_samples = len(x)
    time_centers = []
    ccorr_values = []
    
    for start in range(0, n_samples - window_samples + 1, step_samples):
        end = start + window_samples
        
        # Extract window
        z_x_win = z_x[start:end]
        z_y_win = z_y[start:end]
        
        if orthogonalize:
            # Orthogonalize
            phase_x = np.angle(z_x_win)
            phase_y = np.angle(z_y_win)
            
            y_orth = np.imag(z_y_win * np.exp(-1j * phase_x))
            x_orth = np.imag(z_x_win * np.exp(-1j * phase_y))
            
            env_x = np.abs(z_x_win)
            env_y_orth = np.abs(y_orth)
            env_y = np.abs(z_y_win)
            env_x_orth = np.abs(x_orth)
            
            # Symmetric
            corr1 = compute_envelope_correlation_from_envelopes(env_x, env_y_orth)
            corr2 = compute_envelope_correlation_from_envelopes(env_y, env_x_orth)
            ccorr = (corr1 + corr2) / 2
        else:
            env_x = np.abs(z_x_win)
            env_y = np.abs(z_y_win)
            ccorr = compute_envelope_correlation_from_envelopes(env_x, env_y)
        
        time_centers.append((start + end) / 2 / fs)
        ccorr_values.append(ccorr)
    
    return np.array(time_centers), np.array(ccorr_values)

In [ ]:
# Visualization 10: Time-resolved CCorr

np.random.seed(42)
fs = 500.0
duration = 20.0
t = np.arange(0, duration, 1/fs)
n_samples = len(t)

# Create signals with time-varying amplitude coupling
# First half: high coupling, second half: low coupling
shared_mod = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
indep_mod_x = 1 + 0.5 * np.sin(2 * np.pi * 0.4 * t + 1.0)
indep_mod_y = 1 + 0.5 * np.sin(2 * np.pi * 0.5 * t + 2.0)

# Coupling strength varies over time
coupling = np.ones(n_samples)
coupling[int(n_samples * 0.3):int(n_samples * 0.7)] = 0.2  # Low coupling in middle

env_x = coupling * shared_mod + (1 - coupling) * indep_mod_x
env_y = coupling * shared_mod + (1 - coupling) * indep_mod_y

x = env_x * np.sin(2 * np.pi * 10 * t) + 0.2 * np.random.randn(n_samples)
y = env_y * np.sin(2 * np.pi * 10 * t + np.pi/3) + 0.2 * np.random.randn(n_samples)

# Compute time-resolved correlation
time_centers, ccorr_ts = compute_envelope_correlation_timeseries(
    x, y, fs, (8, 12), window_sec=2.0, overlap=0.75, orthogonalize=True
)

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Panel 1: Signals
ax1 = axes[0]
x_filt = bandpass_filter(x, fs, (8, 12))
y_filt = bandpass_filter(y, fs, (8, 12))
ax1.plot(t, x_filt, color=COLORS['signal_1'], linewidth=0.5, alpha=0.8, label='Signal X')
ax1.plot(t, y_filt, color=COLORS['signal_2'], linewidth=0.5, alpha=0.8, label='Signal Y')
ax1.set_ylabel('Amplitude', fontsize=11)
ax1.set_title('Filtered Signals (8-12 Hz)', fontsize=12, fontweight='bold')
ax1.legend(loc='upper right')

# Panel 2: Envelopes
ax2 = axes[1]
env_x_plot = extract_envelope(x_filt)
env_y_plot = extract_envelope(y_filt)
ax2.plot(t, env_x_plot, color=COLORS['signal_1'], linewidth=1.5, label='Envelope X')
ax2.plot(t, env_y_plot, color=COLORS['signal_2'], linewidth=1.5, label='Envelope Y')
ax2.set_ylabel('Envelope', fontsize=11)
ax2.set_title('Amplitude Envelopes', fontsize=12, fontweight='bold')
ax2.legend(loc='upper right')

# Panel 3: Time-resolved CCorr
ax3 = axes[2]
ax3.plot(time_centers, ccorr_ts, color=COLORS['highlight'], linewidth=2)
ax3.fill_between(time_centers, 0, ccorr_ts, alpha=0.3, color=COLORS['highlight'])
ax3.axhline(0, color='black', linestyle='--', alpha=0.5)

# Mark the regions
ax3.axvspan(0, duration * 0.3, alpha=0.1, color='green', label='High coupling')
ax3.axvspan(duration * 0.3, duration * 0.7, alpha=0.1, color='red', label='Low coupling')
ax3.axvspan(duration * 0.7, duration, alpha=0.1, color='green')

ax3.set_xlabel('Time (s)', fontsize=11)
ax3.set_ylabel('CCorr', fontsize=11)
ax3.set_title('Time-Resolved Envelope Correlation', fontsize=12, fontweight='bold')
ax3.set_ylim(-0.5, 1)
ax3.legend(loc='upper right')

plt.suptitle('Dynamic Envelope Correlation', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

## Section 9: Envelope Correlation Matrix

For multi-channel analysis, we compute CCorr for all channel pairs.

In [ ]:
def compute_envelope_correlation_matrix(
    data: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    orthogonalize: bool = True
) -> NDArray[np.float64]:
    """
    Compute envelope correlation matrix for all channel pairs.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        EEG data with shape (n_channels, n_samples)
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
    orthogonalize : bool, optional
        Whether to use orthogonalization (default: True)
        
    Returns
    -------
    NDArray[np.float64]
        Correlation matrix with shape (n_channels, n_channels)
    """
    n_channels = data.shape[0]
    
    # Filter all channels
    data_filt = bandpass_filter(data, fs, band)
    
    # Analytic signals
    z_data = signal.hilbert(data_filt, axis=-1)
    phases = np.angle(z_data)
    envelopes = np.abs(z_data)
    
    # Initialize matrix
    ccorr_matrix = np.eye(n_channels)  # Diagonal = 1
    
    for i in range(n_channels):
        for j in range(i + 1, n_channels):
            if orthogonalize:
                # Orthogonalize j with respect to i
                j_orth = np.imag(z_data[j] * np.exp(-1j * phases[i]))
                env_j_orth = np.abs(j_orth)
                corr1 = compute_envelope_correlation_from_envelopes(envelopes[i], env_j_orth)
                
                # Orthogonalize i with respect to j
                i_orth = np.imag(z_data[i] * np.exp(-1j * phases[j]))
                env_i_orth = np.abs(i_orth)
                corr2 = compute_envelope_correlation_from_envelopes(envelopes[j], env_i_orth)
                
                ccorr = (corr1 + corr2) / 2
            else:
                ccorr = compute_envelope_correlation_from_envelopes(envelopes[i], envelopes[j])
            
            ccorr_matrix[i, j] = ccorr
            ccorr_matrix[j, i] = ccorr
    
    return ccorr_matrix


def compute_envelope_correlation_matrix_bands(
    data: NDArray[np.float64],
    fs: float,
    bands: dict[str, tuple[float, float]] | None = None,
    orthogonalize: bool = True
) -> dict[str, NDArray[np.float64]]:
    """
    Compute envelope correlation matrices for multiple frequency bands.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        EEG data with shape (n_channels, n_samples)
    fs : float
        Sampling frequency in Hz
    bands : dict[str, tuple[float, float]], optional
        Dictionary of frequency bands (default: standard EEG bands)
    orthogonalize : bool, optional
        Whether to use orthogonalization (default: True)
        
    Returns
    -------
    dict[str, NDArray[np.float64]]
        Dictionary mapping band names to correlation matrices
    """
    if bands is None:
        bands = {
            'theta': (4, 8),
            'alpha': (8, 13),
            'beta': (13, 30),
            'gamma': (30, 45)
        }
    
    return {name: compute_envelope_correlation_matrix(data, fs, band, orthogonalize) 
            for name, band in bands.items()}

In [ ]:
# Generate multi-channel data
np.random.seed(42)
n_channels = 8
n_samples = 5000
fs = 500.0
t = np.arange(n_samples) / fs

# Create data with amplitude coupling structure
data = np.zeros((n_channels, n_samples))

# Shared modulation for cluster 1 (channels 0-2)
shared_mod_1 = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
for i in range(3):
    phase = np.random.uniform(0, 2*np.pi)
    data[i] = shared_mod_1 * np.sin(2 * np.pi * 10 * t + phase) + 0.2 * np.random.randn(n_samples)

# Shared modulation for cluster 2 (channels 3-5)
shared_mod_2 = 1 + 0.5 * np.sin(2 * np.pi * 0.4 * t + 1.5)
for i in range(3, 6):
    phase = np.random.uniform(0, 2*np.pi)
    data[i] = shared_mod_2 * np.sin(2 * np.pi * 10 * t + phase) + 0.2 * np.random.randn(n_samples)

# Independent channels (6-7)
for i in range(6, 8):
    indep_mod = 1 + 0.5 * np.sin(2 * np.pi * (0.3 + 0.1*i) * t + np.random.uniform(0, 2*np.pi))
    phase = np.random.uniform(0, 2*np.pi)
    data[i] = indep_mod * np.sin(2 * np.pi * 10 * t + phase) + 0.3 * np.random.randn(n_samples)

# Compute matrices
ccorr_std = compute_envelope_correlation_matrix(data, fs, (8, 12), orthogonalize=False)
ccorr_orth = compute_envelope_correlation_matrix(data, fs, (8, 12), orthogonalize=True)

In [ ]:
# Visualization 11: Standard vs orthogonalized matrices

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Standard CCorr
ax1 = axes[0]
im1 = ax1.imshow(ccorr_std, cmap='RdBu_r', vmin=-1, vmax=1, aspect='equal')
ax1.set_title('Standard Envelope Correlation', fontsize=12, fontweight='bold')
ax1.set_xlabel('Channel', fontsize=11)
ax1.set_ylabel('Channel', fontsize=11)
ax1.set_xticks(range(n_channels))
ax1.set_yticks(range(n_channels))

# Cluster separators
for pos in [2.5, 5.5]:
    ax1.axhline(pos, color='white', linewidth=1)
    ax1.axvline(pos, color='white', linewidth=1)

plt.colorbar(im1, ax=ax1, shrink=0.8, label='CCorr')

# Orthogonalized CCorr
ax2 = axes[1]
im2 = ax2.imshow(ccorr_orth, cmap='RdBu_r', vmin=-1, vmax=1, aspect='equal')
ax2.set_title('Orthogonalized Envelope Correlation', fontsize=12, fontweight='bold')
ax2.set_xlabel('Channel', fontsize=11)
ax2.set_ylabel('Channel', fontsize=11)
ax2.set_xticks(range(n_channels))
ax2.set_yticks(range(n_channels))

for pos in [2.5, 5.5]:
    ax2.axhline(pos, color='white', linewidth=1)
    ax2.axvline(pos, color='white', linewidth=1)

plt.colorbar(im2, ax=ax2, shrink=0.8, label='CCorr-orth')

plt.suptitle('Envelope Correlation Matrix: Standard vs Orthogonalized\n'
             'Cluster 1: ch 0-2, Cluster 2: ch 3-5, Independent: ch 6-7', 
             fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

## Section 10: Envelope Correlation for Hyperscanning

**Inter-brain envelope correlation** asks: Do participants' brain activity levels co-fluctuate?

This is a fundamentally different question from phase synchrony. High inter-brain amplitude correlation might reflect:
- Shared emotional engagement
- Co-activation patterns during joint attention
- Shared arousal states

**Important**: There's no volume conduction between brains! So orthogonalization is less critical for **between-brain** connectivity (though still useful for **within-brain** blocks).

In [ ]:
def compute_envelope_correlation_hyperscanning(
    data_p1: NDArray[np.float64],
    data_p2: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    orthogonalize_within: bool = True,
    orthogonalize_between: bool = False
) -> dict[str, NDArray[np.float64]]:
    """
    Compute envelope correlation matrices for hyperscanning.
    
    Parameters
    ----------
    data_p1 : NDArray[np.float64]
        Data from participant 1, shape (n_channels, n_samples)
    data_p2 : NDArray[np.float64]
        Data from participant 2, shape (n_channels, n_samples)
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
    orthogonalize_within : bool, optional
        Orthogonalize within-brain pairs (default: True)
    orthogonalize_between : bool, optional
        Orthogonalize between-brain pairs (default: False)
        
    Returns
    -------
    dict[str, NDArray[np.float64]]
        Dictionary containing within_p1, within_p2, between, and full matrices
    """
    n_ch1 = data_p1.shape[0]
    n_ch2 = data_p2.shape[0]
    
    # Filter and get analytic signals
    data_p1_filt = bandpass_filter(data_p1, fs, band)
    data_p2_filt = bandpass_filter(data_p2, fs, band)
    
    z_p1 = signal.hilbert(data_p1_filt, axis=-1)
    z_p2 = signal.hilbert(data_p2_filt, axis=-1)
    
    phases_p1 = np.angle(z_p1)
    phases_p2 = np.angle(z_p2)
    envs_p1 = np.abs(z_p1)
    envs_p2 = np.abs(z_p2)
    
    # Within P1
    within_p1 = np.eye(n_ch1)
    for i in range(n_ch1):
        for j in range(i + 1, n_ch1):
            if orthogonalize_within:
                j_orth = np.abs(np.imag(z_p1[j] * np.exp(-1j * phases_p1[i])))
                i_orth = np.abs(np.imag(z_p1[i] * np.exp(-1j * phases_p1[j])))
                c1 = compute_envelope_correlation_from_envelopes(envs_p1[i], j_orth)
                c2 = compute_envelope_correlation_from_envelopes(envs_p1[j], i_orth)
                ccorr = (c1 + c2) / 2
            else:
                ccorr = compute_envelope_correlation_from_envelopes(envs_p1[i], envs_p1[j])
            within_p1[i, j] = ccorr
            within_p1[j, i] = ccorr
    
    # Within P2
    within_p2 = np.eye(n_ch2)
    for i in range(n_ch2):
        for j in range(i + 1, n_ch2):
            if orthogonalize_within:
                j_orth = np.abs(np.imag(z_p2[j] * np.exp(-1j * phases_p2[i])))
                i_orth = np.abs(np.imag(z_p2[i] * np.exp(-1j * phases_p2[j])))
                c1 = compute_envelope_correlation_from_envelopes(envs_p2[i], j_orth)
                c2 = compute_envelope_correlation_from_envelopes(envs_p2[j], i_orth)
                ccorr = (c1 + c2) / 2
            else:
                ccorr = compute_envelope_correlation_from_envelopes(envs_p2[i], envs_p2[j])
            within_p2[i, j] = ccorr
            within_p2[j, i] = ccorr
    
    # Between participants
    between = np.zeros((n_ch1, n_ch2))
    for i in range(n_ch1):
        for j in range(n_ch2):
            if orthogonalize_between:
                j_orth = np.abs(np.imag(z_p2[j] * np.exp(-1j * phases_p1[i])))
                i_orth = np.abs(np.imag(z_p1[i] * np.exp(-1j * phases_p2[j])))
                c1 = compute_envelope_correlation_from_envelopes(envs_p1[i], j_orth)
                c2 = compute_envelope_correlation_from_envelopes(envs_p2[j], i_orth)
                ccorr = (c1 + c2) / 2
            else:
                ccorr = compute_envelope_correlation_from_envelopes(envs_p1[i], envs_p2[j])
            between[i, j] = ccorr
    
    # Full matrix
    n_total = n_ch1 + n_ch2
    full = np.zeros((n_total, n_total))
    full[:n_ch1, :n_ch1] = within_p1
    full[n_ch1:, n_ch1:] = within_p2
    full[:n_ch1, n_ch1:] = between
    full[n_ch1:, :n_ch1] = between.T
    
    return {
        'within_p1': within_p1,
        'within_p2': within_p2,
        'between': between,
        'full': full
    }


def compute_global_envelope_correlation_hyperscanning(
    ccorr_dict: dict[str, NDArray[np.float64]]
) -> dict[str, float]:
    """
    Compute summary statistics for hyperscanning envelope correlation.
    
    Parameters
    ----------
    ccorr_dict : dict[str, NDArray[np.float64]]
        Output from compute_envelope_correlation_hyperscanning
        
    Returns
    -------
    dict[str, float]
        Mean values for within_p1, within_p2, and between
    """
    n1 = ccorr_dict['within_p1'].shape[0]
    n2 = ccorr_dict['within_p2'].shape[0]
    
    # Extract upper triangular (excluding diagonal) for within
    within_p1_vals = ccorr_dict['within_p1'][np.triu_indices(n1, k=1)]
    within_p2_vals = ccorr_dict['within_p2'][np.triu_indices(n2, k=1)]
    
    return {
        'mean_within_p1': float(np.mean(within_p1_vals)),
        'mean_within_p2': float(np.mean(within_p2_vals)),
        'mean_between': float(np.mean(ccorr_dict['between']))
    }

In [ ]:
# Generate hyperscanning data
np.random.seed(42)
n_channels_per = 4
n_samples = 5000
fs = 500.0
t = np.arange(n_samples) / fs

# Shared between-brain modulation (weak)
shared_mod = 1 + 0.3 * np.sin(2 * np.pi * 0.2 * t)

# P1 within-brain shared modulation
p1_mod = 1 + 0.4 * np.sin(2 * np.pi * 0.3 * t)
data_p1 = np.zeros((n_channels_per, n_samples))
for i in range(n_channels_per):
    combined_mod = 0.5 * p1_mod + 0.3 * shared_mod + 0.2 * (1 + 0.3 * np.random.randn(n_samples))
    phase = np.random.uniform(0, 2*np.pi)
    data_p1[i] = combined_mod * np.sin(2 * np.pi * 10 * t + phase) + 0.15 * np.random.randn(n_samples)

# P2 within-brain shared modulation
p2_mod = 1 + 0.4 * np.sin(2 * np.pi * 0.35 * t + 1.0)
data_p2 = np.zeros((n_channels_per, n_samples))
for i in range(n_channels_per):
    combined_mod = 0.5 * p2_mod + 0.3 * shared_mod + 0.2 * (1 + 0.3 * np.random.randn(n_samples))
    phase = np.random.uniform(0, 2*np.pi)
    data_p2[i] = combined_mod * np.sin(2 * np.pi * 10 * t + phase) + 0.15 * np.random.randn(n_samples)

# Compute hyperscanning correlation
hyper_ccorr = compute_envelope_correlation_hyperscanning(
    data_p1, data_p2, fs, (8, 12), 
    orthogonalize_within=True, orthogonalize_between=False
)

stats_hyper = compute_global_envelope_correlation_hyperscanning(hyper_ccorr)

In [ ]:
# Visualization 13: Hyperscanning envelope correlation

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Full matrix
ax1 = axes[0]
im1 = ax1.imshow(hyper_ccorr['full'], cmap='RdBu_r', vmin=-1, vmax=1, aspect='equal')

# Block separators
ax1.axhline(n_channels_per - 0.5, color='black', linewidth=2)
ax1.axvline(n_channels_per - 0.5, color='black', linewidth=2)

# Labels
n_total = 2 * n_channels_per
labels = [f'P1-{i}' for i in range(n_channels_per)] + [f'P2-{i}' for i in range(n_channels_per)]
ax1.set_xticks(range(n_total))
ax1.set_yticks(range(n_total))
ax1.set_xticklabels(labels, rotation=45, ha='right')
ax1.set_yticklabels(labels)

# Block annotations
ax1.text(1.5, 1.5, 'Within\nP1', ha='center', va='center', fontsize=9, fontweight='bold')
ax1.text(5.5, 5.5, 'Within\nP2', ha='center', va='center', fontsize=9, fontweight='bold')
ax1.text(5.5, 1.5, 'Between', ha='center', va='center', fontsize=9, fontweight='bold')

ax1.set_title('Full Hyperscanning CCorr Matrix', fontsize=12, fontweight='bold')
plt.colorbar(im1, ax=ax1, shrink=0.8, label='CCorr')

# Summary bar chart
ax2 = axes[1]
categories = ['Within P1', 'Within P2', 'Between']
values = [stats_hyper['mean_within_p1'], stats_hyper['mean_within_p2'], stats_hyper['mean_between']]
colors_bar = [COLORS['subject_1'], COLORS['subject_2'], COLORS['accent']]

bars = ax2.bar(categories, values, color=colors_bar, edgecolor='white', linewidth=2)
for bar, val in zip(bars, values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
             f'{val:.3f}', ha='center', fontsize=11, fontweight='bold')

ax2.set_ylabel('Mean CCorr', fontsize=11)
ax2.set_title('Mean Envelope Correlation by Block', fontsize=12, fontweight='bold')
ax2.set_ylim(0, 0.8)
ax2.axhline(0, color='black', linestyle='-', linewidth=1)

plt.suptitle('Inter-Brain Amplitude Coupling in Hyperscanning', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

## Section 11: Envelope Correlation vs Phase Metrics

Phase and amplitude connectivity capture **different phenomena**:

| Aspect | Phase Metrics (PLV, wPLI) | Amplitude Metrics (CCorr) |
|--------|---------------------------|---------------------------|
| Question | Are oscillations aligned in time? | Do strengths fluctuate together? |
| Timescale | Fast (within-cycle, ms) | Slow (envelope changes, seconds) |
| Mechanism | Timing coordination | Activity level co-modulation |

**They can agree or disagree**:
- Both high: Strong coupling on both dimensions
- High phase, low amplitude: Timing aligned but strengths independent
- High amplitude, low phase: Strengths coupled but timing independent

**Recommendation**: Compute and report both—they provide complementary information!

In [ ]:
def compute_plv_from_phases(
    phase_x: NDArray[np.float64],
    phase_y: NDArray[np.float64]
) -> float:
    """Compute PLV from phases."""
    phase_diff = phase_x - phase_y
    return float(np.abs(np.mean(np.exp(1j * phase_diff))))


def compute_wpli_from_phases(
    phase_x: NDArray[np.float64],
    phase_y: NDArray[np.float64]
) -> float:
    """Compute wPLI from phases."""
    phase_diff = phase_x - phase_y
    sin_diff = np.sin(phase_diff)
    num = np.abs(np.sum(sin_diff))
    den = np.sum(np.abs(sin_diff))
    if den == 0:
        return 0.0
    return float(num / den)


def compare_phase_amplitude_connectivity(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> dict[str, float]:
    """
    Compare phase and amplitude connectivity metrics.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        First signal
    y : NDArray[np.float64]
        Second signal
    fs : float
        Sampling frequency
    band : tuple[float, float]
        Frequency band
        
    Returns
    -------
    dict[str, float]
        PLV, wPLI, CCorr, and CCorr-orth values
    """
    # Filter
    x_filt = bandpass_filter(x, fs, band)
    y_filt = bandpass_filter(y, fs, band)
    
    # Phases
    phase_x = extract_phase(x_filt)
    phase_y = extract_phase(y_filt)
    
    # Phase metrics
    plv = compute_plv_from_phases(phase_x, phase_y)
    wpli = compute_wpli_from_phases(phase_x, phase_y)
    
    # Amplitude metrics
    ccorr = compute_envelope_correlation(x, y, fs, band)
    ccorr_orth = compute_orthogonalized_envelope_correlation(x, y, fs, band)
    
    return {
        'plv': plv,
        'wpli': wpli,
        'ccorr': ccorr,
        'ccorr_orth': ccorr_orth
    }

In [ ]:
# Visualization 14: Phase vs amplitude scatter

np.random.seed(42)
n_pairs = 50
n_samples = 5000
fs = 500.0
band = (8, 12)
t = np.arange(n_samples) / fs

wpli_values = []
ccorr_orth_values = []

for _ in range(n_pairs):
    # Random coupling parameters
    phase_coupling = np.random.uniform(0, 1)
    amp_coupling = np.random.uniform(0, 1)
    
    # Generate signals
    shared_mod = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
    indep_mod_x = 1 + 0.5 * np.sin(2 * np.pi * 0.4 * t + np.random.uniform(0, 2*np.pi))
    indep_mod_y = 1 + 0.5 * np.sin(2 * np.pi * 0.5 * t + np.random.uniform(0, 2*np.pi))
    
    env_x = np.sqrt(amp_coupling) * shared_mod + np.sqrt(1-amp_coupling) * indep_mod_x
    env_y = np.sqrt(amp_coupling) * shared_mod + np.sqrt(1-amp_coupling) * indep_mod_y
    
    # Phase coupling
    base_phase = 2 * np.pi * 10 * t
    phase_x = base_phase
    if phase_coupling < 0.5:
        # Low phase coupling: add phase noise
        phase_y = base_phase + np.cumsum(np.random.randn(n_samples)) * 0.1 / fs
    else:
        # High phase coupling: consistent lag
        phase_y = base_phase + np.pi/4 + 0.1 * np.random.randn(n_samples)
    
    x = env_x * np.sin(phase_x) + 0.2 * np.random.randn(n_samples)
    y = env_y * np.sin(phase_y) + 0.2 * np.random.randn(n_samples)
    
    result = compare_phase_amplitude_connectivity(x, y, fs, band)
    wpli_values.append(result['wpli'])
    ccorr_orth_values.append(result['ccorr_orth'])

fig, ax = plt.subplots(figsize=(10, 8))

ax.scatter(wpli_values, ccorr_orth_values, s=80, alpha=0.6, color=COLORS['signal_1'], edgecolors='white')

# Reference lines
ax.axhline(0.3, color='gray', linestyle='--', alpha=0.5, label='Threshold = 0.3')
ax.axvline(0.3, color='gray', linestyle='--', alpha=0.5)

# Quadrant labels
ax.text(0.15, 0.7, 'High Amplitude\nLow Phase', ha='center', fontsize=10, style='italic')
ax.text(0.7, 0.7, 'High Both', ha='center', fontsize=10, style='italic')
ax.text(0.15, 0.1, 'Low Both', ha='center', fontsize=10, style='italic')
ax.text(0.7, 0.1, 'High Phase\nLow Amplitude', ha='center', fontsize=10, style='italic')

ax.set_xlabel('wPLI (Phase Connectivity)', fontsize=12)
ax.set_ylabel('CCorr-orth (Amplitude Connectivity)', fontsize=12)
ax.set_title('Phase vs Amplitude Connectivity: Different Dimensions\n'
             '(Not necessarily correlated!)', fontsize=14, fontweight='bold')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.legend(loc='lower right')

# Correlation
r_val = np.corrcoef(wpli_values, ccorr_orth_values)[0, 1]
ax.text(0.05, 0.95, f'r = {r_val:.2f}', transform=ax.transAxes, fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Correlation between wPLI and CCorr-orth: r = {r_val:.3f}")
print("\n💡 Phase and amplitude connectivity capture different aspects of neural coupling!")

---

## Section 12: Hands-On Exercises

Practice with envelope correlation!

In [ ]:
# Exercise 1: Basic Envelope Correlation
# Generate two signals with shared amplitude modulation
# Verify high positive correlation

np.random.seed(42)

# YOUR CODE HERE
# 1. Create a shared modulation signal
# 2. Apply it to two carriers with different phases
# 3. Compute CCorr and verify it's high

print("Exercise 1: Basic envelope correlation")

In [ ]:
# Exercise 2: Phase Independence
# Generate signals with correlated envelopes but random phases
# Compute CCorr (should be high) and wPLI (should be low)

np.random.seed(42)

# YOUR CODE HERE
# This demonstrates: amplitude coupling ≠ phase coupling

print("Exercise 2: Phase independence demonstration")

In [ ]:
# Exercise 3: Volume Conduction Demonstration
# Simulate volume conduction
# Compare standard vs orthogonalized CCorr

np.random.seed(42)

# YOUR CODE HERE
# Key demonstration of orthogonalization value

print("Exercise 3: Volume conduction demonstration")

In [ ]:
# Exercise 4: Time-Resolved Analysis
# Create signals where envelope correlation changes over time
# Compute sliding window CCorr

np.random.seed(42)

# YOUR CODE HERE
# Track the dynamics of amplitude coupling

print("Exercise 4: Time-resolved envelope correlation")

In [ ]:
# Exercise 5: CCorr Matrix
# Create 8-channel data with amplitude coupling structure
# Compare standard vs orthogonalized matrices

np.random.seed(42)

# YOUR CODE HERE
# Identify which pairs show biggest difference after orthogonalization

print("Exercise 5: CCorr matrix comparison")

In [ ]:
# Exercise 6: Phase vs Amplitude Dissociation
# Create signals with:
#   Pair A: high phase coupling, low amplitude coupling
#   Pair B: low phase coupling, high amplitude coupling

np.random.seed(42)

# YOUR CODE HERE
# Verify the dissociation between metrics

print("Exercise 6: Phase vs amplitude dissociation")

In [ ]:
# Exercise 7: Hyperscanning Application
# Create two-participant data with between-brain amplitude coupling
# Compare CCorr and wPLI results

np.random.seed(42)

# YOUR CODE HERE
# Note differences in within vs between brain patterns

print("Exercise 7: Hyperscanning envelope correlation")

---

## Summary

### Key Takeaways

1. **Envelope correlation** measures amplitude co-modulation—different from phase coupling!

2. **Formula**: Pearson correlation of amplitude envelopes
   - Range: -1 to +1 (can be negative)
   - Pipeline: filter → Hilbert → envelopes → correlate

3. **Volume conduction problem**: Inflates CCorr (same envelope at multiple electrodes)
   - **Solution**: Orthogonalization removes zero-lag component
   - Orthogonalized CCorr (AEC-c) is more conservative

4. **For hyperscanning**:
   - Captures shared activation/engagement
   - Between-brain: orthogonalization less critical
   - Within-brain: orthogonalization recommended

5. **Report both** phase and amplitude metrics—they provide different information!

---

## Discussion Questions

1. A colleague says "we found high envelope correlation between frontal and parietal regions." What follow-up questions would you ask about their analysis?

2. You find high wPLI but low envelope correlation between two regions. What might this mean neurophysiologically? What about the opposite pattern?

3. Orthogonalization removes zero-lag components. But couldn't there be true instantaneous amplitude coupling? Is orthogonalization always appropriate?

4. Envelope correlation operates on slow fluctuations (envelope changes). What timescale of neural processing might this capture compared to phase synchrony?

5. For hyperscanning, you find strong between-brain envelope correlation in alpha band during a conversation task. Is this "neural coupling" or might there be alternative explanations (e.g., shared arousal from the task)?

---

## Next Steps

Proceed to **H02: Power Correlation** to learn about another amplitude-based connectivity approach!